In [1]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse


os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")
from logger import logger 
from config import cfg

# 如果设置为0.3 
# 240/72 效果很差
# 如果设置为0.2
# 225/163 效果适中

cfg.EDIT_DISTANCE_THRESH_LONG = 0.2
cfg.EDIT_DISTANCE_THRESH_SHORT = 0.2
cfg.LONG_URL_THRESH = 100
cfg.SHORT_URL_THRESH = 20

cfg.MIN_SUBSTRING_SIZE = 12
cfg.SINGLE_REGEX_SIZE = 30
cfg.TOTAL_REGEX_SIZE = 30

from vectorize import make_vectorize
from cluster_engine import make_kmeans_cluster, make_string_distance_cluster
from regex_engine import regex_extract, regex_check, regex_publish
import preprocess
import cluster_engine
import regex_engine

In [2]:
def data_loader(filepath, csv = True, txt = False):
    try:
        if csv:
            df = pd.read_csv(filepath)
            urls = list(df.url)
        if txt:
            with open(filepath, "r") as fd:
                urls = [_.strip() for _ in fd]
        logger.debug("Malware URL Count:\t%d" %len(urls))    
        return urls
    except Exception as e:
        logger.error("%s FILE OPEN ERROR! %s" %(filepath, str(e)))
        sys.exit(0)

In [3]:
urls = data_loader("../../data/sangfor/malware.csv", csv = True, txt = False)

2018-04-09 09:22:20 <ipython-input-2-3401d29b96cd> [line:9] DEBUG	Malware URL Count:	2786


In [4]:
url_domain, url_path, url_query, url_param = preprocess.url_statistic_analysis(urls)
url_path_dict, url_path_counter = preprocess.url_path_statistic_analysis(url_path)

print "only domain", len(url_domain)
print "only path", len(url_path)
print "only query", len(url_query)
print "with params", len(url_param)
print url_path_counter

only domain 0
only path 2064
only query 11
with params 0
{0: 2, 1: 53, 2: 1170, 3: 55, 4: 743, 5: 40, 6: 1}


In [5]:
preprocess.dump_urls(url_path, "../../data/EXP_sangfor_malware/url.csv")

2018-04-08 21:16:45 preprocess.py [line:49] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_sangfor_malware/url.csv
2018-04-08 21:16:45 preprocess.py [line:53] DEBUG	URLs has been dump	../../data/EXP_sangfor_malware/url.csv


In [6]:
df = make_vectorize(url_path, domain = True, path = True, param = False,  output_path = "../../data/EXP_sangfor_malware/vector.csv")

2018-04-08 21:16:53 vectorize.py [line:93] DEBUG	vectorization complete! data shape:	(2075, 256)
2018-04-08 21:16:53 vectorize.py [line:38] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_sangfor_malware/vector.csv
2018-04-08 21:16:54 vectorize.py [line:41] DEBUG	vector has beeen dump	../../data/EXP_sangfor_malware/vector.csv


In [7]:
make_kmeans_cluster(data = df, output_path = '../../data/EXP_sangfor_malware/cluster_kmeams.json')

2018-04-08 21:16:56 cluster_engine.py [line:182] DEBUG	beggin to make k-means cluster, total_data_size:	2075	n_clusters10
2018-04-08 21:16:56 cluster_engine.py [line:193] DEBUG	Calinski-Harabasz Score for	2 Cluster K-means	2060.287050923513
2018-04-08 21:16:56 cluster_engine.py [line:193] DEBUG	Calinski-Harabasz Score for	3 Cluster K-means	2553.592147792382
2018-04-08 21:16:57 cluster_engine.py [line:193] DEBUG	Calinski-Harabasz Score for	4 Cluster K-means	2327.26278201617
2018-04-08 21:16:58 cluster_engine.py [line:193] DEBUG	Calinski-Harabasz Score for	5 Cluster K-means	2157.8345834624465
2018-04-08 21:16:59 cluster_engine.py [line:193] DEBUG	Calinski-Harabasz Score for	6 Cluster K-means	1986.6451929170846
2018-04-08 21:17:00 cluster_engine.py [line:193] DEBUG	Calinski-Harabasz Score for	7 Cluster K-means	1866.194492048483
2018-04-08 21:17:00 cluster_engine.py [line:193] DEBUG	Calinski-Harabasz Score for	8 Cluster K-means	1712.024767424559
2018-04-08 21:17:01 cluster_engine.py [line:

In [8]:
cluster_engine.make_string_distance_cluster(data = '../../data/EXP_sangfor_malware/cluster_kmeams.json', 
                             metric = "distance",  
                             file_path = "../../data/EXP_sangfor_malware/cluster_distance.json")

2018-04-08 21:17:05 cluster_engine.py [line:78] DEBUG	K-means data has been loaded	../../data/EXP_sangfor_malware/cluster_kmeams.json
2018-04-08 21:17:05 cluster_engine.py [line:63] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_sangfor_malware/cluster_distance.json
2018-04-08 21:17:05 cluster_engine.py [line:117] DEBUG	----------------0------------------
2018-04-08 21:17:05 cluster_engine.py [line:128] DEBUG	down05071427.xdown4.com/cx/160624/35/%25E6%2596%25B0%25E4%25BA%25BA%25E6%2595%2599%25E7%2589%2588%25E6%2595%25B0%25E5%25AD%25A6%25E4%25B8%2583%25E5%25B9%25B4%25E7%25BA%25A7%25E4%25B8%258B6.1%25E5%25B9%25B3%25E6%2596%25B9%25E6%25A0%25B9%25E8%25AF%25BE%25E4%25BB%25B6%25284%25E4%25BB%25BD%2529@198_104624.exe
2018-04-08 21:17:05 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/1	TOTAL:300	DONE:1
2018-04-08 21:17:05 cluster_engine.py [line:117] DEBUG	----------------1------------------
2018-04-08 21:17:05 cluster_engine.py [line:128] DEBUG	down05073792.xdown4.com/cx/160624/35/%25E6%25B

2018-04-08 21:17:07 cluster_engine.py [line:117] DEBUG	----------------9------------------
2018-04-08 21:17:07 cluster_engine.py [line:128] DEBUG	url.222bz.com/down/%25E3%2580%258A%25E9%2580%259D%25E8%25A1%2580%25E3%2580%2588%25E5%2590%25B8%25E8%25A1%2580%25E9%25AC%25BC%25E9%25AA%2591%25E5%25A3%25AB%25E3%2580%2589%25E3%2580%258B%25E5%2585%25A8%25E6%2596%2587%2520%25E7%2595%25AA%25E5%25A4%2596@363_58655.exe
2018-04-08 21:17:07 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/1	TOTAL:300	DONE:17
2018-04-08 21:17:07 cluster_engine.py [line:117] DEBUG	----------------10------------------
2018-04-08 21:17:07 cluster_engine.py [line:128] DEBUG	19331.url.9xiazaiqi.com/xiaz/Firefox%25E6%25B5%258F%25E8%25A7%2588%25E5%2599%25A8(%25E7%2581%25AB%25E7%258B%2590%25E4%25B8%25AD%25E5%259B%25BD%25E7%2589%2588%25E4%25B8%258B%25E8%25BD%25BD2018)v60.0.3%25E5%2585%258D%25E5%25AE%2589%25E8%25A3%2585%25E7%25BB%25BF%25E8%2589%25B2%25E7%2589%2588@135_7754.exe
2018-04-08 21:17:07 cluster_engine.py [line:131] 

2018-04-08 21:17:09 cluster_engine.py [line:128] DEBUG	down05072283.xdown4.com/cx/160624/35/%25E4%25BE%25A0%25E7%259B%2597%25E9%25A3%259E%25E8%25BD%25A6%25E7%25BD%25AA%25E6%2581%25B6%25E9%2583%25BD%25E5%25B8%2582%25E4%25B8%25AD%25E6%2596%2587%25E5%25AE%258C%25E5%2585%25A8%25E7%2589%2588%25E6%25B8%25B8%25E6%2588%258F%25E4%25B8%258B%25E8%25BD%25BD@195_706.exe
2018-04-08 21:17:09 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/1	TOTAL:300	DONE:33
2018-04-08 21:17:09 cluster_engine.py [line:117] DEBUG	----------------22------------------
2018-04-08 21:17:09 cluster_engine.py [line:128] DEBUG	19454.url.246546.com/xiaz/%25E7%258B%25A9%25E7%258C%258E%25E5%25AD%25A3%25E8%258A%2582.s02.HuntingSeason.FullSeason2.Uncensored.2015.HD.720p.x264.AAC.mp4.torrent%25E3%2580%2590%25E5%2590%258C%25E6%2580%25A7%25E3%2580%2591%25E7%258B%25A9%25E7%258C%258E%25E5%25AD%25A3%25E8%258A%2582(%25E7%25BE%258E%25E5%259B%25BD)2015_%25E7%2599%25BE%25E5%25BA%25A6%25E7%25BD%2591%25E7%259B%2598-%25E7%25BD%2591%25E7%25

2018-04-08 21:17:10 cluster_engine.py [line:128] DEBUG	down05072070.xdown4.com/cx/160624/35/2017%25E5%25B9%25B4%25E4%25B8%2583%25E5%25B9%25B4%25E7%25BA%25A7%25E4%25B8%258B%25E7%25AC%25AC6%25E7%25AB%25A0%25E4%25B8%2580%25E5%2585%2583%25E4%25B8%2580%25E6%25AC%25A1%25E6%2596%25B9%25E7%25A8%258B%25E7%25BB%25BC%25E5%2590%2588%25E6%25B5%258B%25E8%25AF%2595%25E9%25A2%2598%25E5%2590%25AB%25E7%25AD%2594%25E6%25A1%2588@198_136508.exe
2018-04-08 21:17:10 cluster_engine.py [line:128] DEBUG	down05073791.xdown4.com/cx/160624/35/%25E5%258C%2597%25E5%25B8%2588%25E5%25A4%25A7%25E4%25B8%2583%25E5%25B9%25B4%25E7%25BA%25A7%25E4%25B8%258B%25E5%2586%258C%25E7%25AC%25AC%25E4%25B8%2589%25E7%25AB%25A0%25E5%258F%2598%25E9%2587%258F%25E4%25B9%258B%25E9%2597%25B4%25E7%259A%2584%25E5%2585%25B3%25E7%25B3%25BB%25E5%258D%2595%25E5%2585%2583%25E6%25B5%258B%25E8%25AF%2595%25E9%25A2%2598%25E5%2590%25AB%25E7%25AD%2594%25E6%25A1%2588@198_148589.exe
2018-04-08 21:17:10 cluster_engine.py [line:128] DEBUG	down05071690.xdown4

2018-04-08 21:17:11 cluster_engine.py [line:128] DEBUG	19716.xc.cangpie.com/xiaz/%25E5%25AD%25A6%25E6%25A0%25A1%25E7%259A%2584%25E9%2598%25B6%25E6%25A2%25AF%25E5%2585%25A8%25E9%259B%2586_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25BD%2591%25E7%259B%2598_%25E5%259C%25A8%25E7%25BA%25BF%25E8%25A7%2582%25E7%259C%258B_%25E8%25BF%2585%25E9%259B%25B7%25E4%25B8%258B%25E8%25BD%25BD@439_2.exe
2018-04-08 21:17:11 cluster_engine.py [line:128] DEBUG	19624.xc.cangpie.com/xiaz/%25E6%2588%2598%25E9%25BC%2593%25E6%2593%2582%25E5%2585%25A8%25E9%259B%2586_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25BD%2591%25E7%259B%2598_%25E5%259C%25A8%25E7%25BA%25BF%25E8%25A7%2582%25E7%259C%258B_%25E8%25BF%2585%25E9%259B%25B7%25E4%25B8%258B%25E8%25BD%25BD@439_2.exe
2018-04-08 21:17:11 cluster_engine.py [line:128] DEBUG	19715.xc.gongnou.com/xiaz/%25E5%25A4%25A7%25E5%25AE%258B%25E7%25BB%25AF%25E9%2597%25BB%25E5%25BD%2595%25E9%25AB%2598%25E6%25B8%2585_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25

...

2018-04-08 21:20:29 cluster_engine.py [line:128] DEBUG	down05071499.xdown4.com/cx/160624/35/%25E9%2598%2591%25E5%25B0%25BE%25E7%2582%258E%25E7%25AE%2580%25E4%25BB%258BPPT%25E8%25AF%25BE%25E4%25BB%25B6%25E4%25B8%258B%25E8%25BD%25BD@148_24456.exe
2018-04-08 21:20:29 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1039
2018-04-08 21:20:29 cluster_engine.py [line:117] DEBUG	----------------886------------------
2018-04-08 21:20:29 cluster_engine.py [line:128] DEBUG	19338.url.9xiazaiqi.com/xiaz/CAD%25C3%25D4%25C4%25E3%25BF%25B4%25CD%25BCv2018R1%25B9%25D9%25CD%25F8%25D7%25EE%25D0%25C2%25B0%25E6@136_14914.exe
2018-04-08 21:20:29 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1040
2018-04-08 21:20:29 cluster_engine.py [line:117] DEBUG	----------------890------------------
2018-04-08 21:20:29 cluster_engine.py [line:128] DEBUG	cl.zasuv.com/download/%25E6%2599%25BA%25E8%2583%25BD%25E9%2599%2588%25E6%25A1%25A5%25E4%25BA%2594%25E7%25AC%2594_95@891143.exe


2018-04-08 21:20:32 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1058
2018-04-08 21:20:32 cluster_engine.py [line:117] DEBUG	----------------958------------------
2018-04-08 21:20:33 cluster_engine.py [line:128] DEBUG	cl.zasuv.com/download/%25E8%2585%25BE%25E8%25AE%25AF%25E7%2594%25B5%25E8%2584%2591%25E7%25AE%25A1%25E5%25AE%25B6_95@890831.exe
2018-04-08 21:20:33 cluster_engine.py [line:128] DEBUG	cl.zasuv.com/download/%25E8%2585%25BE%25E8%25AE%25AF%25E6%2589%258B%25E6%25B8%25B8%25E5%258A%25A9%25E6%2589%258B_95@909703.exe
2018-04-08 21:20:33 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1060
2018-04-08 21:20:33 cluster_engine.py [line:117] DEBUG	----------------967------------------
2018-04-08 21:20:33 cluster_engine.py [line:128] DEBUG	url.222bz.com/down/%25E4%25BB%258A%25E6%2597%25A5%25E5%25A4%25B4%25E6%259D%25A1%25E7%2594%25B5%25E8%2584%2591%25E7%2589%2588%2520v6.6.2%25E5%25AE%2598%25E6%2596%25B9pc%25E7%2589%2588@289_50308.exe
2018-04-08

2018-04-08 21:20:36 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1078
2018-04-08 21:20:36 cluster_engine.py [line:117] DEBUG	----------------1018------------------
2018-04-08 21:20:36 cluster_engine.py [line:128] DEBUG	down05073434.xdown4.com/cx/160624/35/%25E8%25B6%25B3%25E7%2590%2583%25E6%2588%2598%25E6%259C%25AF%25E6%259D%25BF%25E8%25BD%25AF%25E4%25BB%25B6@105_1232885.exe
2018-04-08 21:20:36 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1079
2018-04-08 21:20:36 cluster_engine.py [line:117] DEBUG	----------------1021------------------
2018-04-08 21:20:36 cluster_engine.py [line:128] DEBUG	down05073495.xdown4.com/cx/160624/35/%25E5%2588%2586%25E6%2595%25B0%25E9%2599%25A4%25E4%25BB%25A5%25E6%2595%25B4%25E6%2595%25B0ppt%25E8%25AF%25BE%25E4%25BB%25B6@198_127473.exe
2018-04-08 21:20:36 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1080
2018-04-08 21:20:36 cluster_engine.py [line:117] DEBUG	----------------1024----------

2018-04-08 21:20:39 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1097
2018-04-08 21:20:39 cluster_engine.py [line:117] DEBUG	----------------1062------------------
2018-04-08 21:20:40 cluster_engine.py [line:128] DEBUG	19346.url.9xiazaiqi.com/xiaz/%25E6%25B8%25B8%25E8%25AE%25AF%25E6%25B8%25B8%25E6%2588%258F%25E7%259B%2592%25E5%25AD%2590v1.1.9.8%25E5%25AE%2598%25E6%2596%25B9%25E6%25AD%25A3%25E5%25BC%258F%25E7%2589%2588@317_24413.exe
2018-04-08 21:20:40 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1098
2018-04-08 21:20:40 cluster_engine.py [line:117] DEBUG	----------------1064------------------
2018-04-08 21:20:40 cluster_engine.py [line:128] DEBUG	19642.url.9xiazaiqi.com/xiaz/SumatraPDFv3.2(32%2BX64%25E4%25BD%258D%25E5%258F%258C%25E7%2589%2588%25E6%259C%25AC)PDF%25E9%2598%2585%25E8%25AF%25BB%25E5%2599%25A8%25E7%25BB%25BF%25E8%2589%25B2%25E7%2589%2588@464_2.exe
2018-04-08 21:20:40 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1

2018-04-08 21:20:43 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1116
2018-04-08 21:20:43 cluster_engine.py [line:117] DEBUG	----------------1109------------------
2018-04-08 21:20:43 cluster_engine.py [line:128] DEBUG	14614.url.222bz.com/down/%25E7%258E%258B%25E8%2580%2585%25E8%258D%25A3%25E8%2580%2580%25E7%2594%25B5%25E8%2584%2591%25E7%2589%2588@904_4967.exe
2018-04-08 21:20:43 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1117
2018-04-08 21:20:43 cluster_engine.py [line:117] DEBUG	----------------1111------------------
2018-04-08 21:20:43 cluster_engine.py [line:128] DEBUG	xc.wenpie.com/xiaz/%25E4%25BC%25AF%25E7%2588%25B5%25E5%25BA%2584%25E5%259B%25AD%25E7%259A%2584%25E7%25A7%2598%25E5%25AF%25862@802_6892.exe
2018-04-08 21:20:43 cluster_engine.py [line:131] DEBUG	>>>>>>> BATCH:3/3	TOTAL:1125	DONE:1118
2018-04-08 21:20:43 cluster_engine.py [line:117] DEBUG	----------------1112------------------
2018-04-08 21:20:43 cluster_engine.py [line

In [9]:
regex_extract(input_file_path = "../../data/EXP_sangfor_malware/cluster_distance.json", 
                  output_file_path = "../../data/EXP_sangfor_malware/regex_raw.txt")

2018-04-08 21:21:37 regex_engine.py [line:46] DEBUG	Cluster Data has been loaded	../../data/EXP_sangfor_malware/cluster_distance.json
2018-04-08 21:21:37 regex_engine.py [line:254] DEBUG	total cluster num:	1160
2018-04-08 21:21:37 regex_engine.py [line:256] DEBUG	big cluster:	40
2018-04-08 21:21:37 regex_engine.py [line:258] DEBUG	small cluster:	206
2018-04-08 21:21:37 regex_engine.py [line:259] DEBUG	single one:	914
2018-04-08 21:21:37 regex_engine.py [line:260] DEBUG	cluster size detail:	Counter({1: 914, 2: 131, 3: 49, 4: 26, 5: 17, 6: 6, 7: 3, 8: 3, 20: 3, 9: 1, 10: 1, 13: 1, 15: 1, 21: 1, 24: 1, 358: 1, 63: 1})
2018-04-08 21:22:10 regex_engine.py [line:223] DEBUG	19691.xc.cangpie.com/xiaz/%25E4%25BF%25AE%25E5%25A5%25B3%25E7%25A6%2581%25E7%2588%25B1%25E8%25B4%25B9%25E7%25BD%2597%25E7%25BA%25B3%25E7%259A%2584%25E4%25BF%25AE%25E5%25A5%25B3%25E4%25BB%25AC%25E9%25AB%2598%25E6%25B8%2585_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25BD%2591%25E7%259B%2598_%25E5%259C%25A8%25E7%25BA%

2018-04-08 21:22:41 regex_engine.py [line:223] DEBUG	19650.xc.cangpie.com/xiaz/%25E4%25B8%2580%25E5%25A4%259C%25E6%2583%2585%25E6%25B7%25B1%25E9%25AB%2598%25E6%25B8%2585_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25BD%2591%25E7%259B%2598_%25E5%259C%25A8%25E7%25BA%25BF%25E8%25A7%2582%25E7%259C%258B_%25E8%25BF%2585%25E9%259B%25B7%25E4%25B8%258B%25E8%25BD%25BD@439_2.exe
2018-04-08 21:22:41 regex_engine.py [line:223] DEBUG	19700.xc.cangpie.com/xiaz/%25E7%258B%25AC%25E7%25AB%258B%25E6%2597%25A5%25E9%25AB%2598%25E6%25B8%2585_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25BD%2591%25E7%259B%2598_%25E5%259C%25A8%25E7%25BA%25BF%25E8%25A7%2582%25E7%259C%258B_%25E8%25BF%2585%25E9%259B%25B7%25E4%25B8%258B%25E8%25BD%25BD@439_2.exe
2018-04-08 21:22:41 regex_engine.py [line:223] DEBUG	19700.xc.cangpie.com/xiaz/%25E9%25A2%2590%25E5%2592%258C%25E5%259B%25AD%25E9%25AB%2598%25E6%25B8%2585_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25BD%2591%25E7%259B%2598_%25E5%259C%25A8%25E7%25BA%25

2018-04-08 21:22:41 regex_engine.py [line:223] DEBUG	19646.xc.cangpie.com/xiaz/%25E5%25B9%25B4%25E8%25BD%25BB%25E7%259A%2584%25E5%25B2%25B3%25E6%25AF%258D%25E9%25AB%2598%25E6%25B8%2585_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25BD%2591%25E7%259B%2598_%25E5%259C%25A8%25E7%25BA%25BF%25E8%25A7%2582%25E7%259C%258B_%25E8%25BF%2585%25E9%259B%25B7%25E4%25B8%258B%25E8%25BD%25BD@439_2.exe
2018-04-08 21:22:41 regex_engine.py [line:223] DEBUG	19720.xc.cangpie.com/xiaz/%25E7%258B%2597%25E5%258D%2581%25E4%25B8%2589/%25E7%258B%259713%25E9%25AB%2598%25E6%25B8%2585_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25BD%2591%25E7%259B%2598_%25E5%259C%25A8%25E7%25BA%25BF%25E8%25A7%2582%25E7%259C%258B_%25E8%25BF%2585%25E9%259B%25B7%25E4%25B8%258B%25E8%25BD%25BD@439_2.exe
2018-04-08 21:22:41 regex_engine.py [line:227] DEBUG	index:	1	['\\.com/xiaz/%25E', '_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25BD%2591%25E7%259B%2598_%25E5%259C%25A8%25E7%25BA%25BF%25E8%25A7%2582%25E7%259C%258B_%25E

2018-04-08 21:23:02 regex_engine.py [line:223] DEBUG	19696.xc.gongnou.com/xiaz/%25E4%25B9%259D%25E9%25A6%2596%25E6%25AD%258C/%25E6%2583%2585%25E6%25AC%25B2%25E4%25B9%259D%25E6%25AD%258C%25E9%25AB%2598%25E6%25B8%2585_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25BD%2591%25E7%259B%2598_%25E5%259C%25A8%25E7%25BA%25BF%25E8%25A7%2582%25E7%259C%258B_%25E8%25BF%2585%25E9%259B%25B7%25E4%25B8%258B%25E8%25BD%25BD@439_2.exe
2018-04-08 21:23:02 regex_engine.py [line:223] DEBUG	19713.xc.cangpie.com/xiaz/%25E7%258B%2597%25E5%258D%2581%25E4%25B8%2589/%25E7%258B%259713%25E9%25AB%2598%25E6%25B8%2585_%25E7%2599%25BE%25E5%25BA%25A6%25E4%25BA%2591%25E7%25BD%2591%25E7%259B%2598_%25E5%259C%25A8%25E7%25BA%25BF%25E8%25A7%2582%25E7%259C%258B_%25E8%25BF%2585%25E9%259B%25B7%25E4%25B8%258B%25E8%25BD%25BD@439_2.exe
2018-04-08 21:23:02 regex_engine.py [line:223] DEBUG	19716.xc.cangpie.com/xiaz/%25E4%25B8%2589%25E4%25B8%25AA%25E5%25AE%25B6%25E4%25BC%2599%25E9%25AB%2598%25E6%25B8%2585_%25E7%2599%25BE%25E5%25BA

2018-04-08 21:24:16 regex_engine.py [line:223] DEBUG	19456.url.9xiazaiqi.com/xiaz/%25E7%25BE%258E%25E7%25AF%2587%25E7%2594%25B5%25E8%2584%2591%25E7%2589%2588v4.3.3%25E5%25AE%2598%25E6%2596%25B9PC%25E7%2589%2588@135_54776.exe
2018-04-08 21:24:16 regex_engine.py [line:223] DEBUG	19426.url.9xiazaiqi.com/xiaz/%25E7%25BE%258E%25E7%25AF%2587%25E7%2594%25B5%25E8%2584%2591%25E7%2589%2588v4.3.3%25E5%25AE%2598%25E6%2596%25B9PC%25E7%2589%2588@317_29867.exe
2018-04-08 21:24:16 regex_engine.py [line:223] DEBUG	19355.url.246546.com/xiaz/%25E7%25BE%258E%25E7%25AF%2587%25E7%2594%25B5%25E8%2584%2591%25E7%2589%2588v4.3.3%25E5%25AE%2598%25E6%2596%25B9PC%25E7%2589%2588@316_54776.exe
2018-04-08 21:24:16 regex_engine.py [line:223] DEBUG	19601.url.9xiazaiqi.com/xiaz/%25E7%25BE%258E%25E7%25AF%2587%25E7%2594%25B5%25E8%2584%2591%25E7%2589%2588v4.3.3%25E5%25AE%2598%25E6%2596%25B9PC%25E7%2589%2588@317_29867.exe
2018-04-08 21:24:16 regex_engine.py [line:227] DEBUG	index:	3	['\\.com/xiaz/%25E', '%25E7%2594%25B5%25E

2018-04-08 21:24:32 regex_engine.py [line:223] DEBUG	cr173.down.123ch.cn/download/%25E6%258A%2596%25E9%259F%25B3%25E7%259F%25AD%25E8%25A7%2586%25E9%25A2%2591%25E7%2594%25B5%25E8%2584%2591%25E7%2589%2588_51@427938.exe
2018-04-08 21:24:32 regex_engine.py [line:223] DEBUG	cr173.down.123ch.cn/download/%25E8%2593%259D%25E5%25A2%25A8%25E4%25BA%2591%25E7%258F%25AD%25E8%25AF%25BE%25E7%2594%25B5%25E8%2584%2591%25E7%2589%2588_51@286753.exe
2018-04-08 21:24:32 regex_engine.py [line:223] DEBUG	cr173.down.123ch.cn/download/%25E8%25B6%2585%25E6%2598%259F%25E5%25AD%25A6%25E4%25B9%25A0%25E9%2580%259A%25E7%2594%25B5%25E8%2584%2591%25E7%2589%2588_51@331054.exe
2018-04-08 21:24:32 regex_engine.py [line:223] DEBUG	cr173.down.123ch.cn/download/%25E6%2598%2586%25E4%25BB%2591%25E9%2593%25B6%25E8%25A1%258C%25E7%25BD%2591%25E9%2593%25B6%25E5%258A%25A9%25E6%2589%258B_51@117562.exe
2018-04-08 21:24:32 regex_engine.py [line:223] DEBUG	cr173.down.123ch.cn/download/%25E6%258A%2596%25E9%259F%25B3%25E7%259F%25AD%25E8

2018-04-08 21:24:54 regex_engine.py [line:275] DEBUG	extract regex count:	225
2018-04-08 21:24:54 regex_engine.py [line:276] DEBUG	extract regex time cost:	196.411909


In [10]:
regex_check(input_file_path = "../../data/EXP_sangfor_malware/regex_raw.txt", 
            test_benign_file_path = "../../data/sangfor/safe_plus.csv",
            test_malicious_file_path = "../../data/EXP_sangfor_malware/url.csv",
            result_file_path = "../../data/EXP_sangfor_malware/regex_result.txt", 
            n_jobs = 8)

2018-04-08 21:25:03 regex_engine.py [line:60] DEBUG	Regex Data has been loaded	../../data/EXP_sangfor_malware/regex_raw.txt
2018-04-08 21:25:05 regex_engine.py [line:71] DEBUG	Test Data has been loaded	../../data/sangfor/safe_plus.csv
2018-04-08 21:25:05 regex_engine.py [line:71] DEBUG	Test Data has been loaded	../../data/EXP_sangfor_malware/url.csv


('batch index', 0, 'sample index', 0, 'FP', 0, 'TP', 8)
('batch index', 1, 'sample index', 0, 'FP', 7, 'TP', 2)
('batch index', 0, 'sample index', 1, 'FP', 0, 'TP', 21)
('batch index', 2, 'sample index', 0, 'FP', 0, 'TP', 9)
('batch index', 2, 'sample index', 1, 'FP', 0, 'TP', 2)
('batch index', 1, 'sample index', 1, 'FP', 0, 'TP', 3)
('batch index', 0, 'sample index', 2, 'FP', 0, 'TP', 4)
('batch index', 2, 'sample index', 2, 'FP', 0, 'TP', 10)
('batch index', 3, 'sample index', 0, 'FP', 0, 'TP', 2)
('batch index', 0, 'sample index', 3, 'FP', 0, 'TP', 2)
('batch index', 3, 'sample index', 1, 'FP', 0, 'TP', 2)
('batch index', 2, 'sample index', 3, 'FP', 0, 'TP', 4)
('batch index', 3, 'sample index', 2, 'FP', 1, 'TP', 6)
('batch index', 1, 'sample index', 2, 'FP', 0, 'TP', 1)
('batch index', 4, 'sample index', 0, 'FP', 1, 'TP', 7)
('batch index', 2, 'sample index', 4, 'FP', 0, 'TP', 2)
('batch index', 3, 'sample index', 3, 'FP', 1, 'TP', 4)
('batch index', 5, 'sample index', 0, 'FP', 0,

('batch index', 5, 'sample index', 19, 'FP', 0, 'TP', 5)
('batch index', 1, 'sample index', 13, 'FP', 0, 'TP', 1)
('batch index', 7, 'sample index', 13, 'FP', 0, 'TP', 2)
('batch index', 4, 'sample index', 17, 'FP', 6, 'TP', 135)
('batch index', 3, 'sample index', 25, 'FP', 0, 'TP', 2)
('batch index', 0, 'sample index', 14, 'FP', 0, 'TP', 1)
('batch index', 5, 'sample index', 20, 'FP', 0, 'TP', 3)
('batch index', 7, 'sample index', 14, 'FP', 0, 'TP', 2)
('batch index', 4, 'sample index', 18, 'FP', 0, 'TP', 2)
('batch index', 6, 'sample index', 16, 'FP', 0, 'TP', 3)
('batch index', 3, 'sample index', 26, 'FP', 6, 'TP', 135)
('batch index', 5, 'sample index', 21, 'FP', 10, 'TP', 2)
('batch index', 3, 'sample index', 27, 'FP', 0, 'TP', 2)
('batch index', 6, 'sample index', 17, 'FP', 1, 'TP', 2)
('batch index', 7, 'sample index', 15, 'FP', 0, 'TP', 3)
('batch index', 1, 'sample index', 14, 'FP', 0, 'TP', 2)
('batch index', 4, 'sample index', 19, 'FP', 1, 'TP', 8)
('batch index', 5, 'sample

2018-04-08 21:27:42 regex_engine.py [line:318] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_sangfor_malware/regex_result.txt
2018-04-08 21:27:42 regex_engine.py [line:323] DEBUG	Check Result has been dump	../../data/EXP_sangfor_malware/regex_result.txt


In [11]:
regex_publish(result_file_path= "../../data/EXP_sangfor_malware/regex_result.txt",
              publish_file_path="../../data/EXP_sangfor_malware/regex_publish.txt",
              publish_fp_thresh = 0,
              publish_tp_thresh = 1)

2018-04-08 21:27:45 regex_engine.py [line:338] DEBUG	check Data has been loaded	../../data/EXP_sangfor_malware/regex_result.txt
2018-04-08 21:27:45 regex_engine.py [line:347] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_sangfor_malware/regex_publish.txt
2018-04-08 21:27:45 regex_engine.py [line:352] DEBUG	Publish Result Count:	163
2018-04-08 21:27:45 regex_engine.py [line:353] DEBUG	Publish Result has been dump	../../data/EXP_sangfor_malware/regex_publish.txt
